In [1]:
import pandas as pd
import json
import os
from IPython.display import display, Markdown
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from collections import Counter

In [5]:
with open('./results/thresholds_IQR.json') as json_data:
    thresholds = json.load(json_data)
thresholds

flattened = {}

for text_id, categories in thresholds.items():
    for category, features in categories.items():
        for feature, value in features.items():
            index = f"{category}/{feature}"
            flattened.setdefault(index, {})[text_id] = value

df = pd.DataFrame.from_dict(flattened, orient='index')
df = df[['N1', 'N2', 'N3', 'N4']]  # ensure column order

df

,N1,N2,N3,N4
sentence-token-level/max_size_aux_verbs,5.0000,3.5000,2.0000,2.0000
sentence-token-level/max_size_passive,0.0000,5.0000,5.0000,5.0000
sentence-token-level/max_size_named_entities,7.0000,9.5000,9.5000,7.0000
sentence-token-level/max_size_np_pp_modifiers,29.5000,16.5000,16.0000,15.6250
sentence-token-level/max_size_subordination,21.5000,31.5000,49.0000,53.2500
sentence-token-level/max_size_coordination,3.5000,6.0000,6.0000,6.0000
document-token-level/total_token_ratio_aux_verbs,0.1623,0.1130,0.1019,0.0639
document-token-level/total_token_ratio_passive,0.0000,0.0220,0.0220,0.0211
document-token-level/total_token_ratio_named_entities,0.4142,0.2401,0.1955,0.1570
document-token-level/total_token_ratio_subordination,0.5601,0.6977,0.8173,0.7142


In [3]:
def draw_boxplot(data, title="Boxplot", xlabel="Values"):
    """
    Draw a boxplot for the given data.

    Parameters:
        data (list or array): The input values.
        title (str): Title of the plot.
        xlabel (str): Label for the x-axis.
    """
    plt.figure(figsize=(6, 4))
    plt.boxplot(data, vert=False, patch_artist=True,
                boxprops=dict(facecolor="lightblue", color="blue"),
                medianprops=dict(color="red", linewidth=2),
                whiskerprops=dict(color="blue"),
                capprops=dict(color="blue"),
                flierprops=dict(marker="o", color="orange", alpha=0.7))

    plt.title(title)
    plt.xlabel(xlabel)
    plt.show()